# Original Multi-step prediction

In [ ]:
from init import *
init()
import tensorflow as tf

import os
import pandas as pd
from mll.LSTMPredictor import LSTMPredictor

In [ ]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)
df = pd.read_csv(csv_path)

predictor = LSTMPredictor(df[['p (mbar)', 'T (degC)', 'rho (g/m**3)']], 'T (degC)', 300000, 720, 72, step=6, seed=13)
model = predictor.multi_step_model
history = predictor.multi_step_history

In [ ]:
for i, (x, y) in enumerate( predictor.validation_dataset.take(5) ):
    if i > 2:
        multi_step_plot(x[0], y[0], model.predict(x)[0], predictor.step, predictor.dependent_variable)

In [ ]:
from utils import plot_df
from ipywidgets import interact, IntSlider
max_count = 20
dfs = predictor.multi_step_dfs(max_count)
fig = plot_df(dfs[0])

@interact(step=IntSlider(min=1, max=max_count, step=1, value=1))
def update(step):
    global dfs
    with fig.batch_update():
        df = dfs[step-1]
        fig.data[0].y = df['True Future'].values
        fig.data[1].y = df['Predicted Future'].values
fig

# CME & Quandl

In [ ]:
import mll.QuandlFeed as qf
#CME_CL is CME WTI crude oil (38) and CME_NG is Natural Gas (43)
df = qf.get_data({'CHRIS/CME_' + k: k for k in ['CL1', 'CL2', 'CL3', 'CL4', 'CL5','NG1', 'NG2', 'NG3', 'NG4', 'NG5']}
                 , 'Last')
df.plot()

In [ ]:
#df.iloc[1760:][['CL3','NG1','NG2','NG3','NG4','NG5']]
df2 = df[['CL3','NG1','NG2','NG3','NG4','NG5']].dropna()
df2['CL3'] /= 10
df2[0:100].plot()

In [ ]:
df2[df2.NG1.isna()]